## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделите выборку на обучающую/проверочную в соотношении 80/20.

Построим параллельный ансамбль (бэггинг) решающих деревьев, используя случайный лес.

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing

### Загрузка данных

In [2]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")
print (data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 128 entries, Id to Response
dtypes: float64(18), int64(109), object(1)
memory usage: 58.0+ MB
None


### Предобработка данных

In [3]:
data["Product_Info_2_1"] = data["Product_Info_2"].str.slice(0, 1)
data["Product_Info_2_2"] = pd.to_numeric(data["Product_Info_2"].str.slice(1, 2))
data.drop(labels=["Product_Info_2"], axis=1, inplace=True)
for l in data["Product_Info_2_1"].unique():
    data["Product_Info_2_1" + l] = data["Product_Info_2_1"].isin([l]).astype("int8")
data.drop(labels=["Product_Info_2_1"], axis=1, inplace=True)
data.fillna(value=-1, inplace=True)

### Набор столбцов для расчета

In [4]:
columns_groups = ["Insurance_History", "InsurеdInfo", "Medical_Keyword",
                  "Family_Hist", "Medical_History", "Product_Info"]
columns = ["Wt", "Ht", "Ins_Age", "BMI"]
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print (columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [5]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(pd.DataFrame(data,
                                                     columns=columns)))
columns_transformed = data_transformed.columns
data_transformed["Response"] = data["Response"]

### Оптимизация памяти

In [6]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        else:
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [7]:
data_transformed = reduce_mem_usage(data_transformed)
print (data_transformed.info())

Потребление памяти меньше на 40.49 Мб (минус 75.1 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB
None


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [8]:
data_train, data_test = train_test_split(data_transformed,
                                         test_size=0.2)
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)
print (data_train.head())

              0         1         2         3         4         5         6  \
49512 -0.513672 -0.710449 -0.391602 -0.178589 -1.634766 -0.169434  0.862305   
4432  -0.866211 -0.220581 -1.981445 -0.935547  0.611816 -0.169434 -1.159180   
6493  -1.688477 -1.445312 -0.845703 -1.436523 -1.634766 -0.169434  0.862305   
42525 -1.359375 -0.465576  0.213989 -1.457031  0.611816  5.902344  0.862305   
12794  0.308838 -0.220581 -0.164429  0.578613  0.611816 -0.169434 -1.159180   

              7         8         9  ...       109       110       111  \
49512 -1.013672  0.864258 -0.928711  ... -0.083679  0.441650 -0.149292   
4432   1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   
6493  -1.013672  0.863770 -0.928711  ... -0.083679 -2.263672 -0.149292   
42525  0.043671  0.861328 -0.928711  ... -0.083679  0.441650 -0.149292   
12794  1.100586 -1.156250  1.130859  ... -0.083679  0.441650 -0.149292   

            112       113       114       115       116       117  Response  
49

### Перекрестная проверка случайного леса
Каждое дерево (по умолчанию, их 100) строится на своей части выборки со своим набором параметров (max_features). Решение принимается путем голосования деревьев.

Например, 10 деревьев для 1 строки (кортежа) исходных параметров дали следующие классы и их вероятности:

{1:0.5, 1:0.8, 2:0.9, 3:0.7, 5:0.5, 1:0.4, 2:0.5, 6:0.5, 3:0.4, 1:0.95}

По итогам голосования выбирается самый популярный класс, это 1 в данном случае.

Если в случайном лесу слишком много деревьев, то точность предсказания будет меньше, чем у одного, полностью обученного дерева. Число деревьев (estimators) должно соответствовать количеству классов в предсказании (class), размеру выборки (N) и числу разбиений (fold). Примерная формула:
estimators = N / (20-100) / fold / class

В нашем случае, N=60000, fold=5, class=8 => estimators=15...75

In [9]:
x = pd.DataFrame(data_train, columns=columns_transformed)
model = RandomForestClassifier(random_state=17, n_estimators=77,
                max_depth=17, max_features=27, min_samples_leaf=30)

Диапазон тестирования параметров модели ограничен только вычислительной мощностью. Для проверки модели имеет смысл провести индивидуальные перекрестные проверки для каждого параметра в отдельности, затем в итоговой проверке перепроверить самые лучшие найденные параметры с отклонением +/-10%.

In [11]:
tree_params = {
    'max_depth': range(15,17),
    'max_features': range(26,28),
    'n_estimators': range(75,77),
    'min_samples_leaf': range(19,21)
}
tree_grid = GridSearchCV(model, tree_params, cv=5, n_jobs=2,
            verbose=True, scoring=make_scorer(cohen_kappa_score))
tree_grid.fit(x, data_train["Response"])

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  9.3min
[Parallel(n_jobs=2)]: Done  80 out of  80 | elapsed: 16.2min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=17,
                                              max_features=27,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=30,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=77, n_jobs=None,
                                              oob_score=False, random_state=17,
                                        

Выведем самые оптимальные параметры и построим итоговую модель

In [13]:
print (tree_grid.best_params_)
model = RandomForestClassifier(random_state=17,
        min_samples_leaf=tree_grid.best_params_['min_samples_leaf'],
        max_features=tree_grid.best_params_['max_features'],
        max_depth=tree_grid.best_params_['max_depth'],
        n_estimators=tree_grid.best_params_['n_estimators'])
model.fit(x, data_train["Response"])

{'max_depth': 15, 'max_features': 27, 'min_samples_leaf': 19, 'n_estimators': 75}


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features=27,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=19, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=75,
                       n_jobs=None, oob_score=False, random_state=17, verbose=0,
                       warm_start=False)

### Предсказание данных и оценка модели

In [14]:
x_test = pd.DataFrame(data_test, columns=columns_transformed)
data_test["target"] = model.predict(x_test)

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3

In [15]:
print ("Случайный лес:",
      round(cohen_kappa_score(data_test["target"],
            data_test["Response"], weights="quadratic"), 3))

Случайный лес: 0.487


### Матрица неточностей

In [16]:
print ("Случайный лес\n",
      confusion_matrix(data_test["target"], data_test["Response"]))

Случайный лес
 [[ 115   63   11   18   12   32    8    1]
 [ 192  303   12    0   68   66    6    4]
 [  20   20   52   10    1    2    0    0]
 [  42   38   51  189    0    2    0    3]
 [  95  161   22    0  581   92   14    6]
 [ 321  272   25   33  219 1234  343  133]
 [ 148  132    2    8   56  270  614  132]
 [ 324  308    7   42  122  524  648 3648]]
